# **4. Modelado**


## **1. Configuración inicial para modelado supervisado**

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import joblib
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_auc_score, f1_score, accuracy_score, confusion_matrix, classification_report
)
from sklearn.ensemble import RandomForestClassifier

# Modelos gradiente
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Rutas
ROOT = Path("..")
DATA = ROOT / "data" / "processed"
MODELS = ROOT / "models"
MODELS.mkdir(parents=True, exist_ok=True)

# Cargar datasets procesados de la Sección 3
X_train = pd.read_parquet(DATA/"X_train.parquet")
X_test  = pd.read_parquet(DATA/"X_test.parquet")
y_train = pd.read_parquet(DATA/"y_train.parquet")["Churn"].astype("int8")
y_test  = pd.read_parquet(DATA/"y_test.parquet")["Churn"].astype("int8")

# Cargar preprocessor (ColumnTransformer)
preprocessor = joblib.load(ROOT/"models"/"preprocessor.joblib")

# Pos_weight para clases desbalanceadas (para XGB/LGBM)
pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()
pos_weight = float(np.clip(pos_weight, 1.0, 20.0))  # acotar por seguridad
print("scale_pos_weight (aprox):", pos_weight)

# CV y scoring
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {"roc_auc":"roc_auc", "f1":"f1", "accuracy":"accuracy"}


scale_pos_weight (aprox): 2.768561872909699


Ese valor (~2.7) indica que la clase Churn = 1 es unas 2.7 veces menos frecuente que la clase No Churn = 0, por lo que los modelos de gradiente deberán darle mayor peso a los errores en la clase positiva.

## **2. Generación de *splits* crudos (data/interim) para Telco Churn**

In [4]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split

ROOT = Path("..")
DATA = ROOT / "data"
PROC = DATA / "processed"
INTERIM = DATA / "interim"
INTERIM.mkdir(parents=True, exist_ok=True)

TARGET = "Churn"
ID_COL = "customerID"

# 1) Carga del parquet limpio del EDA
df_raw = pd.read_parquet(PROC / "telco_churn.parquet")

# 2) Normalizaciones mínimas
# 2.1 Target a 0/1 (robusto a mayúsculas/acentos/espacios)
t = df_raw[TARGET].astype(str).str.strip().str.lower()
mapa = {
    "yes": 1, "no": 0,
    "sí": 1, "si": 1,
    "true": 1, "false": 0, "0": 0, "1": 1
}
df_raw[TARGET] = t.map(mapa)

# Si quedó algo sin mapear, avisa para inspección
resto = df_raw.loc[df_raw[TARGET].isna(), TARGET]
if len(resto):
    print("Valores no mapeados en Churn ->", df_raw.loc[df_raw[TARGET].isna(), TARGET].unique())
    # Si sabes que los NaN equivalen a 'No', puedes descomentar:
    # df_raw[TARGET] = df_raw[TARGET].fillna(0)

df_raw[TARGET] = df_raw[TARGET].astype("int8")

# 2.2 TotalCharges a numérico (en este dataset suele haber strings vacíos)
if "TotalCharges" in df_raw.columns:
    df_raw["TotalCharges"] = pd.to_numeric(df_raw["TotalCharges"], errors="coerce")

# 3) X/y crudos (sin procesar)
X_raw = df_raw.drop(columns=[TARGET])
y_raw = df_raw[TARGET]

# 4) Split estratificado
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    X_raw, y_raw,
    test_size=0.20, random_state=42, stratify=y_raw
)

# 5) Guardar para la sección de modelado (pipeline con preprocessor)
X_train_raw.to_parquet(INTERIM/"X_train_raw.parquet", index=False)
X_test_raw.to_parquet(INTERIM/"X_test_raw.parquet", index=False)
y_train_raw.to_frame(TARGET).to_parquet(INTERIM/"y_train_raw.parquet", index=False)
y_test_raw.to_frame(TARGET).to_parquet(INTERIM/"y_test_raw.parquet", index=False)

print("OK: splits crudos guardados en data/interim/")
print("Distribución y_train:", y_train_raw.value_counts(normalize=True).round(3).to_dict())
print("Distribución y_test :", y_test_raw.value_counts(normalize=True).round(3).to_dict())



OK: splits crudos guardados en data/interim/
Distribución y_train: {0: 0.735, 1: 0.265}
Distribución y_test : {0: 0.735, 1: 0.265}


Esas proporciones indican que aproximadamente el 26.5 % de los clientes hacen churn y el 73.5 % se mantiene, tanto en entrenamiento como en prueba, gracias al stratify=y_raw.

In [5]:
from pathlib import Path
import pandas as pd
import joblib

ROOT = Path("..")
INTERIM = ROOT / "data" / "interim"
MODELS = ROOT / "models"

# Carga splits crudos
X_train = pd.read_parquet(INTERIM/"X_train_raw.parquet")
X_test  = pd.read_parquet(INTERIM/"X_test_raw.parquet")
y_train = pd.read_parquet(INTERIM/"y_train_raw.parquet")["Churn"].astype("int8")
y_test  = pd.read_parquet(INTERIM/"y_test_raw.parquet")["Churn"].astype("int8")

# Carga tu preprocessor.joblib (hecho en sección 3)
preprocessor = joblib.load(MODELS/"preprocessor.joblib")

print("Shapes:", X_train.shape, X_test.shape)


Shapes: (5634, 20) (1409, 20)


## **3. Resultados de la búsqueda de hiperparámetros para Random Forest**

In [6]:
rf_pipe = Pipeline(steps=[
    ("pre", preprocessor),
    ("clf", RandomForestClassifier(
        n_estimators=400,
        class_weight="balanced",   # ayuda con desbalance
        random_state=42,
        n_jobs=-1
    ))
])

rf_grid = {
    "clf__max_features": ["sqrt", "log2"],
    "clf__min_samples_split": [2, 5, 10],
    "clf__min_samples_leaf": [1, 2, 4],
    "clf__bootstrap": [True],
    "clf__criterion": ["gini", "entropy"]  # también 'log_loss' en nuevas versiones
}

rf_search = GridSearchCV(
    rf_pipe, rf_grid, cv=cv, scoring=scoring, refit="roc_auc",
    n_jobs=-1, verbose=1
)
rf_search.fit(X_train, y_train)

print("RF best ROC-AUC:", rf_search.best_score_)
print("RF best params:", rf_search.best_params_)

# Guardar
joblib.dump(rf_search.best_estimator_, MODELS/"rf_best.joblib")
pd.DataFrame(rf_search.cv_results_).to_csv(MODELS/"rf_cv_results.csv", index=False)


Fitting 5 folds for each of 36 candidates, totalling 180 fits
RF best ROC-AUC: 0.8442564977030317
RF best params: {'clf__bootstrap': True, 'clf__criterion': 'gini', 'clf__max_features': 'log2', 'clf__min_samples_leaf': 4, 'clf__min_samples_split': 2}


La salida muestra el resumen del `GridSearchCV` aplicado al pipeline con **RandomForestClassifier**:

- `Fitting 5 folds for each of 36 candidates, totalling 180 fits`  
  - Se definió una **búsqueda en malla** (`GridSearchCV`) con:
    - 36 combinaciones distintas de hiperparámetros (`36 candidates`).
    - Validación cruzada estratificada de **5 folds**.  
  - En total se entrenaron **180 modelos** (36 combinaciones × 5 folds).

- `RF best ROC-AUC: 0.8442564977030317`  
  - El mejor modelo encontrado alcanzó un **ROC-AUC ≈ 0.844**, lo que indica una **buena capacidad de discriminación** entre clientes que hacen churn (1) y los que no (0).  
  - Mientras más se acerque a 1.0, mejor separa las clases; valores > 0.8 suelen considerarse bastante buenos en muchos problemas reales.

- `RF best params: {...}`  
  El mejor modelo usa los siguientes **hiperparámetros**:

  - `clf__bootstrap: True`  
    El Random Forest se entrena con *bootstrap* (muestreo con reemplazo) para cada árbol, lo típico en RF estándar.

  - `clf__criterion: 'gini'`  
    La impureza de los nodos se mide con el índice de Gini (en vez de `entropy`).  
    Esto afecta cómo se hacen las particiones en los árboles.

  - `clf__max_features: 'log2'`  
    En cada split, el árbol considera solo `log2(n_features)` variables candidatas.  
    Esto introduce más aleatoriedad y puede mejorar generalización.

  - `clf__min_samples_leaf: 4`  
    Cada hoja del árbol debe tener al menos **4 observaciones**.  
    Hojas con mínimo mayor reducen el sobreajuste (evitan hojas muy pequeñas).

  - `clf__min_samples_split: 2`  
    Un nodo se puede dividir si tiene al menos **2 muestras**.  
    Combinado con `min_samples_leaf=4`, permite cierta profundidad pero sigue controlando el tamaño mínimo de hojas.

En resumen, de todas las combinaciones probadas, el modelo con estos hiperparámetros fue el que logró **mayor ROC-AUC media en CV (≈ 0.844)**

## **4. Resultados de la búsqueda de hiperparámetros para XGBoost (XGBClassifier)**

In [7]:
xgb_pipe = Pipeline(steps=[
    ("pre", preprocessor),
    ("clf", XGBClassifier(
        n_estimators=600,
        learning_rate=0.05,
        objective="binary:logistic",
        eval_metric="auc",
        tree_method="hist",      # 'gpu_hist' si tienes GPU
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    ))
])

xgb_grid = {
    "clf__max_depth": [3, 5, 7],
    "clf__min_child_weight": [1, 3, 5],
    "clf__gamma": [0, 0.5, 1.0],
    "clf__reg_alpha": [0, 0.5, 1.0],
    "clf__reg_lambda": [1.0, 2.0],
    "clf__scale_pos_weight": [1.0, pos_weight],  # maneja desbalance
    "clf__subsample": [0.7, 0.9],
    "clf__colsample_bytree": [0.7, 0.9]
}

xgb_search = GridSearchCV(
    xgb_pipe, xgb_grid, cv=cv, scoring=scoring, refit="roc_auc",
    n_jobs=-1, verbose=1
)
xgb_search.fit(X_train, y_train)

print("XGB best ROC-AUC:", xgb_search.best_score_)
print("XGB best params:", xgb_search.best_params_)

joblib.dump(xgb_search.best_estimator_, MODELS/"xgb_best.joblib")
pd.DataFrame(xgb_search.cv_results_).to_csv(MODELS/"xgb_cv_results.csv", index=False)


Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
XGB best ROC-AUC: 0.848440533786993
XGB best params: {'clf__colsample_bytree': 0.7, 'clf__gamma': 1.0, 'clf__max_depth': 3, 'clf__min_child_weight': 3, 'clf__reg_alpha': 1.0, 'clf__reg_lambda': 2.0, 'clf__scale_pos_weight': 1.0, 'clf__subsample': 0.9}


La salida corresponde al `GridSearchCV` aplicado al pipeline con **XGBClassifier**:

- `Fitting 5 folds for each of 1296 candidates, totalling 6480 fits`  
  - El grid de hiperparámetros tiene **1.296 combinaciones distintas** (`1296 candidates`).  
  - Se utilizó validación cruzada estratificada de **5 folds**, por lo que se entrenaron:  
    \[
    1296 combinaciones  folds = 6480 
    \]
  - Esto implica una búsqueda mucho más exhaustiva que la de Random Forest, por eso el número de *fits* es tan alto.

- `XGB best ROC-AUC: 0.848440533786993`  
  - El mejor modelo de XGBoost alcanzó un **ROC-AUC ≈ 0.848**, ligeramente mejor que el obtenido con Random Forest (~0.844).  
  - Un ROC-AUC en este rango indica que el modelo tiene **muy buena capacidad para separar clientes que hacen churn (1) de los que no (0)**, midiendo el trade-off entre TPR y FPR a través de diferentes umbrales.

- `XGB best params: {...}`  
  Los hiperparámetros óptimos encontrados son:

  - `clf__colsample_bytree: 0.7`  
    Cada árbol solo ve el **70 % de las variables** en cada split, lo que introduce aleatoriedad y ayuda a evitar sobreajuste.

  - `clf__gamma: 1.0`  
    Controla el **mínimo descenso en la pérdida** requerido para hacer una nueva partición.  
    Un valor mayor (1.0) hace que el árbol sea más conservador, evitando splits que no mejoren suficientemente el modelo.

  - `clf__max_depth: 3`  
    Profundidad máxima de los árboles = 3.  
    Árboles poco profundos suelen **generalizar mejor** y reducen el riesgo de sobreajuste.

  - `clf__min_child_weight: 3`  
    Peso mínimo de las instancias en un nodo hijo.  
    Un valor 3 también hace que el modelo sea más conservador, evitando hojas con muy pocas observaciones.

  - `clf__reg_alpha: 1.0`  
    **Regularización L1** distinta de cero, que induce cierta sparsidad en los pesos y ayuda a controlar el sobreajuste.

  - `clf__reg_lambda: 2.0`  
    **Regularización L2** relativamente fuerte, penalizando coeficientes grandes y contribuyendo a la estabilidad del modelo.

  - `clf__scale_pos_weight: 1.0`  
    Aunque probaste incluir el `pos_weight` teórico para el desbalance, la búsqueda encontró que el mejor desempeño es con **peso 1.0**, es decir, sin reponderar adicionalmente la clase positiva en XGBoost (probablemente porque el desbalance no es extremo o ya está manejado indirectamente).

  - `clf__subsample: 0.9`  
    Cada árbol se entrena con el **90 % de las filas** (submuestreo de instancias), lo que introduce variabilidad y ayuda a mejorar la generalización.



## **5. Resultados de la búsqueda de hiperparámetros con LightGBM**

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from joblib import Memory
from pathlib import Path

# 5 particiones estratificadas (requisito)
cv5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {"roc_auc":"roc_auc","f1":"f1","accuracy":"accuracy"}

# cachea el preprocesamiento para no rehacer OHE/imputación en cada fold
cache_dir = Path("../.cache_sklearn"); cache_dir.mkdir(exist_ok=True)
pipe_lgb = Pipeline(
    steps=[("pre", preprocessor),
           ("clf", LGBMClassifier(
               n_estimators=500,
               learning_rate=0.05,
               objective="binary",
               random_state=42,
               n_jobs=-1
           ))],
    memory=Memory(cache_dir)
)

# Grid FINAL pequeño (12–24 combinaciones aprox). Ajusta si quieres 16–20 como tope.
param_grid_final = {
    "clf__num_leaves": [31, 63],
    "clf__min_child_samples": [20, 50],
    "clf__reg_alpha": [0.0, 0.5],
    "clf__reg_lambda": [0.0, 0.5],
    "clf__subsample": [0.8],
    "clf__colsample_bytree": [0.8],
    "clf__class_weight": [None, "balanced"],  # ayuda con desbalance
    "clf__max_bin": [255],                    # fijo para acelerar
}

search5 = GridSearchCV(
    estimator=pipe_lgb,
    param_grid=param_grid_final,
    cv=cv5,
    scoring=scoring,
    refit="roc_auc",
    n_jobs=-1,
    verbose=1
)
search5.fit(X_train, y_train)
print("Best ROC-AUC (cv=5):", search5.best_score_)
print("Best params (cv=5):", search5.best_params_)




Fitting 5 folds for each of 32 candidates, totalling 160 fits
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(ColumnTransformer(transformers=[('num', SimpleImputer(strategy='median'),
                                 ['SeniorCitizen', 'tenure', 'MonthlyCharges',
                                  'TotalCharges']),
                                ('cat',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['gender', 'Partner', 'Dependents',
                                  'PhoneService', 'MultipleLines',
                                  'InternetService', 'OnlineSecurity',
                                  'OnlineBackup', 'DeviceProtection',
                                  'TechSupport', 'StreamingTV',
                                  'StreamingMovies', 'C

 Resumen de la búsqueda

- `Fitting 5 folds for each of 32 candidates, totalling 160 fits`  
  - El grid de hiperparámetros tiene **32 combinaciones diferentes**.  
  - Con validación cruzada estratificada de **5 folds**, se entrenan:  
    \[
    32 \text{ modelos} \times 5 \text{ folds} = 160 \text{ entrenamientos}
    \]
  - Esto cumple el requisito de **5 particiones estratificadas**.



 Mensajes de `Memory` y del Pipeline

- El bloque largo que comienza con:
  - `[Memory] Calling sklearn.pipeline._fit_transform_one...`  
    indica que el `Pipeline` está usando **caché** (`memory=Memory(cache_dir)`):  
    - El preprocesador (`ColumnTransformer` con imputación + OneHotEncoder) se ejecuta una vez y se guarda en disco.  
    - En los siguientes folds de la CV no tiene que recalcular todo el OHE/imputación desde cero, lo que **acelera bastante** la búsqueda.

- Dentro de ese mensaje se ven:
  - El `ColumnTransformer` con:
    - Numéricas: `SeniorCitizen`, `tenure`, `MonthlyCharges`, `TotalCharges` (imputadas con mediana).  
    - Categóricas: todas las variables de tipo categoría codificadas con `OneHotEncoder(handle_unknown='ignore')`.  
  - También se listan algunas filas de ejemplo del dataset de entrada y el vector `y` (la columna `Churn`).



 Logs de LightGBM

Los mensajes tipo:

- `[LightGBM] [Info] Number of positive: 1495, number of negative: 4139`  
  - Muestran el **nº de observaciones de la clase positiva y negativa** en el conjunto de entrenamiento (desbalance moderado).

- `[LightGBM] [Info] Total Bins 667`  
  - Cantidad total de *bins* generados internamente para discretizar las features.

- `[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 45`  
  - Se están usando **5634 filas** y **45 variables** luego de todo el preprocesamiento (OHE incluido).

- `[LightGBM] [Info] Start training from score 0.000000`  
  - LightGBM inicializa el modelo asumiendo un score base (probabilidad media inicial de la clase positiva).

Estos mensajes son puramente informativos y confirman que el modelo está entrenando correctamente.



 Métrica principal obtenida

- `Best ROC-AUC (cv=5): 0.8339218758605339`  
  - El mejor modelo encontrado con el grid actual alcanza un **ROC-AUC ≈ 0.834** promedio en validación cruzada de 5 folds.  
  - Esto indica una **buena capacidad discriminativa**, aunque algo por debajo del XGBoost (~0.848) y del Random Forest (~0.844) que entrenaste antes.  
  - Aun así, es un modelo competitivo y sirve como **tercer benchmark fuerte**.



 Mejores hiperparámetros encontrados

- `Best params (cv=5): {'clf__class_weight': 'balanced', 'clf__colsample_bytree': 0.8, 'clf__max_bin': 255, 'clf__min_child_samples': 20, 'clf__num_leaves': 31, 'clf__reg_alpha': 0.5, 'clf__reg_lambda': 0.5, 'clf__subsample': 0.8}`  

Interpretación de cada uno:

- `clf__class_weight: 'balanced'`  
  - LightGBM **pondera más la clase minoritaria (Churn=1)** para compensar el desbalance (1495 vs 4139).  
  - Esto suele mejorar métricas como F1 y recall de la clase positiva.

- `clf__num_leaves: 31`  
  - Número máximo de hojas en cada árbol.  
  - 31 es un valor moderado: árboles relativamente simples → **mejor generalización y menos sobreajuste**.

- `clf__min_child_samples: 20`  
  - Mínimo de ejemplos que debe tener una hoja.  
  - Obliga a que las hojas tengan al menos 20 muestras, haciendo el modelo más conservador.

- `clf__reg_alpha: 0.5` y `clf__reg_lambda: 0.5`  
  - **Regularización L1 (`alpha`) y L2 (`lambda`)** distintas de cero.  
  - Ambas penalizan la complejidad del modelo y ayudan a controlar el sobreajuste.

- `clf__subsample: 0.8`  
  - Cada árbol se entrena con un **80 % de las filas**, introduciendo aleatoriedad tipo *bagging*, lo que mejora la robustez del modelo.

- `clf__colsample_bytree: 0.8`  
  - Cada árbol ve sólo el **80 % de las variables** ⇒ reduce correlación entre árboles y también ayuda a la generalización.

- `clf__max_bin: 255`  
  - Número máximo de bins para discretizar las features.  
  - 255 es el valor típico por defecto y se fijó para **acelerar** el entrenamiento.


## **6. Verificación de rutas del proyecto y modelos disponibles**

In [11]:
from pathlib import Path

CWD = Path.cwd()
# Si corres dentro de docs/, sube a la raíz del proyecto:
PROJECT = CWD if CWD.name != "docs" else CWD.parent
MODELS = PROJECT / "models"
MODELS.mkdir(parents=True, exist_ok=True)

print("CWD:", CWD)
print("PROJECT:", PROJECT)
print("MODELS:", MODELS)
print("Modelos disponibles:", list(MODELS.glob("*.joblib")))


CWD: c:\Users\juana\MLOPS\miniproyecto6\docs
PROJECT: c:\Users\juana\MLOPS\miniproyecto6
MODELS: c:\Users\juana\MLOPS\miniproyecto6\models
Modelos disponibles: [WindowsPath('c:/Users/juana/MLOPS/miniproyecto6/models/preprocessor.joblib'), WindowsPath('c:/Users/juana/MLOPS/miniproyecto6/models/rf_best.joblib'), WindowsPath('c:/Users/juana/MLOPS/miniproyecto6/models/xgb_best.joblib')]


In [12]:
import joblib, pandas as pd

print("Best ROC-AUC (cv=5):", search5.best_score_)
print("Best params (cv=5):", search5.best_params_)

# 2.1) Guardar el MEJOR PIPELINE (preprocesador + lgbm)
lgbm_best_path = MODELS / "lgbm_best.joblib"
joblib.dump(search5.best_estimator_, lgbm_best_path)

# 2.2) Guardar resultados de la CV
cv_path = MODELS / "lgbm_cv_results.csv"
pd.DataFrame(search5.cv_results_).to_csv(cv_path, index=False)

# 2.3) (Opcional) Guardar SOLO el clasificador ya ajustado
clf_only_path = MODELS / "lgbm_clf_only.joblib"
best_clf = search5.best_estimator_.named_steps["clf"]
joblib.dump(best_clf, clf_only_path)

print("Guardados:")
print(" -", lgbm_best_path)
print(" -", cv_path)
print(" -", clf_only_path)
print("Ahora hay:", list(MODELS.glob('*.joblib')))


Best ROC-AUC (cv=5): 0.8339218758605339
Best params (cv=5): {'clf__class_weight': 'balanced', 'clf__colsample_bytree': 0.8, 'clf__max_bin': 255, 'clf__min_child_samples': 20, 'clf__num_leaves': 31, 'clf__reg_alpha': 0.5, 'clf__reg_lambda': 0.5, 'clf__subsample': 0.8}
Guardados:
 - c:\Users\juana\MLOPS\miniproyecto6\models\lgbm_best.joblib
 - c:\Users\juana\MLOPS\miniproyecto6\models\lgbm_cv_results.csv
 - c:\Users\juana\MLOPS\miniproyecto6\models\lgbm_clf_only.joblib
Ahora hay: [WindowsPath('c:/Users/juana/MLOPS/miniproyecto6/models/lgbm_best.joblib'), WindowsPath('c:/Users/juana/MLOPS/miniproyecto6/models/lgbm_clf_only.joblib'), WindowsPath('c:/Users/juana/MLOPS/miniproyecto6/models/preprocessor.joblib'), WindowsPath('c:/Users/juana/MLOPS/miniproyecto6/models/rf_best.joblib'), WindowsPath('c:/Users/juana/MLOPS/miniproyecto6/models/xgb_best.joblib')]


## **7. Comparación final de modelos en el conjunto de prueba (RF vs XGB vs LGBM)**

In [13]:
candidates = {
    "rf": joblib.load(MODELS/"rf_best.joblib"),
    "xgb": joblib.load(MODELS/"xgb_best.joblib"),
    "lgbm": joblib.load(MODELS/"lgbm_best.joblib"),
}

scores = {}
for name, model in candidates.items():
    prob = model.predict_proba(X_test)[:,1]
    pred = (prob >= 0.5).astype(int)
    scores[name] = {
        "roc_auc": roc_auc_score(y_test, prob),
        "f1": f1_score(y_test, pred),
        "accuracy": accuracy_score(y_test, pred)
    }
pd.DataFrame(scores).T


,roc_auc,f1,accuracy
rf,0.839673,0.624277,0.769340
xgb,0.846081,0.579666,0.803407
lgbm,0.829939,0.611111,0.761533


Interpretación

- **XGBoost (xgb)** es tu mejor modelo global:  
  - Tiene el **ROC-AUC más alto (0.846)** → mejor capacidad de separar clientes que harán *churn*.  
  - Tiene también la **mejor accuracy (0.803)**.  
  - Su F1 es inferior a RF pero aceptable, considerando el desbalance.

- **Random Forest (rf)** muestra:
  - Buen F1 (el mejor de los tres)  
  - Pero menor ROC-AUC y accuracy en comparación con XGB.

- **LightGBM (lgbm)**:
  - Es el que peor se desempeña en este caso.  
  - Aunque no está mal, queda por detrás de XGB y RF en todas las métricas.



###  Conclusión final

**El mejor modelo para tu caso de Churn es XGBoost**, ya que:
- Generaliza mejor (mejor ROC-AUC y accuracy en test)
- Maneja bien el desbalance
- Logra la mejor separación entre clases

## **8. Evaluación final del mejor modelo (XGBoost) en el conjunto de prueba**

In [14]:
# Escoge el mejor por ROC-AUC y muestra reporte
best_name = max(scores, key=lambda k: scores[k]["roc_auc"])
best_model = candidates[best_name]
print("Mejor en test:", best_name, scores[best_name])

y_prob = best_model.predict_proba(X_test)[:,1]
y_pred = (y_prob >= 0.5).astype(int)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["No Churn","Churn"], columns=["Pred No","Pred Sí"])
cm_df


Mejor en test: xgb {'roc_auc': np.float64(0.8460810147510915), 'f1': 0.5796661608497724, 'accuracy': 0.8034066713981547}

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.91      0.87      1035
           1       0.67      0.51      0.58       374

    accuracy                           0.80      1409
   macro avg       0.75      0.71      0.73      1409
weighted avg       0.79      0.80      0.79      1409



,Pred No,Pred Sí
No Churn,941,94
Churn,183,191



 Clase 0 (No Churn)
- **Precision 0.84:** de los usuarios predichos como *No Churn*, el 84% realmente no se va.  
- **Recall 0.91:** captura correctamente el 91% de los clientes que NO hacen churn.  
- **F1 0.87:** excelente equilibrio.

Esto significa que el modelo es **muy bueno identificando clientes que se quedan**.


- **Precision 0.67:** cuando el modelo dice que alguien hará churn, acierta un 67%.  
- **Recall 0.51:** solo identifica correctamente al 51% de los clientes que sí se irán.  
- **F1 0.58:** rendimiento moderado.

Esto refleja la dificultad del problema:  
**es más difícil detectar a los clientes que sí se van**, típico en datasets desbalanceados.
 Aciertos:
- **941** No Churn clasificados correctamente.
- **191** Churn clasificados correctamente.

 Errores:
- **94** clientes que se quedan, el modelo los marcó como churn (falsos positivos).  
- **183** clientes que sí se van, el modelo NO los detectó (falsos negativos).

 **Los falsos negativos (183)** son especialmente importantes para empresas,  
porque representan clientes que sí abandonan pero el modelo no los alerta.

Conclusión general

El modelo XGBoost:

- Es **excelente identificando a los clientes que no se van** (Recall = 0.91).
- Tiene **buen poder de separación global** (ROC-AUC = 0.846).
- **Detecta alrededor de la mitad de los churn reales**, lo cual es razonable pero puede mejorarse ajustando el threshold o aplicando técnicas adicionales (SMOTE, focal loss, calibration, cost-sensitive learning).

Este es un **resultado sólido y típico** para un problema real de churn con desbalance.


In [15]:
# === Paths (ajústalos si ya los tienes definidos) ===
from pathlib import Path
import joblib
import json

ROOT = Path.cwd()
MODELS = ROOT / "models"
MODELS.mkdir(exist_ok=True)

# 1) Extraer el MEJOR PIPELINE del grid
best_pipe = xgb_search.best_estimator_     # <--- Pipeline(pre=..., clf=...)

# 2) Separar artefactos
preprocessor = best_pipe.named_steps["pre"]
clf_only     = best_pipe.named_steps["clf"]

# 3) Guardar por separado (sobrescribe si ya existen)
joblib.dump(preprocessor, MODELS / "preprocessor.joblib")
joblib.dump(clf_only,     MODELS / "xgb_clf_only.joblib")

# (opcional pero MUY útil) guardar nombres de features transformadas
feat_names = list(preprocessor.get_feature_names_out())
with open(MODELS / "feature_names.json", "w", encoding="utf-8") as f:
    json.dump(feat_names, f, ensure_ascii=False, indent=2)

print("✓ Guardados:")
print(" -", MODELS / "preprocessor.joblib")
print(" -", MODELS / "xgb_clf_only.joblib")
print(" -", MODELS / "feature_names.json")


✓ Guardados:
 - c:\Users\juana\MLOPS\miniproyecto6\docs\models\preprocessor.joblib
 - c:\Users\juana\MLOPS\miniproyecto6\docs\models\xgb_clf_only.joblib
 - c:\Users\juana\MLOPS\miniproyecto6\docs\models\feature_names.json


In [20]:
# ======== Inference mínimo con preprocessor + xgb_clf_only ========
from pathlib import Path
import json
import joblib
import numpy as np
import pandas as pd

# ---------- util: encontrar la carpeta del proyecto (busca 'models/') ----------
def find_project_root(marker="models", max_up=6):
    p = Path.cwd()
    for _ in range(max_up):
        if (p/marker).exists():
            return p
        p = p.parent
    raise FileNotFoundError(f"No pude encontrar la carpeta '{marker}' subiendo desde {Path.cwd()}")

PROJECT_ROOT = find_project_root("models")
MODELS_DIR   = (PROJECT_ROOT / "models").resolve()
print("PROJECT_ROOT:", PROJECT_ROOT)
print("MODELS_DIR  :", MODELS_DIR)

# ---------- carga de artefactos ----------
def load_artifacts():
    pre = joblib.load(MODELS_DIR / "preprocessor.joblib")      # ColumnTransformer/Pipeline de features
    clf = joblib.load(MODELS_DIR / "xgb_clf_only.joblib")      # clasificador ya entrenado sobre el espacio transformado

    # columnas crudas que el preprocesador espera (del fit)
    try:
        raw_cols = pre.feature_names_in_.tolist()
    except Exception:
        # fallback: si por alguna razón no existe, intenta leer de JSON (opcional)
        raw_cols = None

    # nombres de features transformadas (opcional, para trazas/depuración)
    try:
        with open(MODELS_DIR / "feature_names.json", "r", encoding="utf-8") as f:
            transformed_names = json.load(f)
    except Exception:
        transformed_names = None

    return pre, clf, raw_cols, transformed_names

preprocessor, clf, RAW_COLS, TRANSFORMED_NAMES = load_artifacts()
print("RAW_COLS (del preprocessor):", len(RAW_COLS) if RAW_COLS is not None else None)
print("TRANSFORMED_NAMES:", len(TRANSFORMED_NAMES) if TRANSFORMED_NAMES else None)

# ---------- util: normalizar entrada a DataFrame crudo con el orden correcto ----------
def to_aligned_dataframe(data):
    """
    data: dict (un registro), list[dict] (varios), o DataFrame crudo.
    Devuelve un DataFrame con las columnas en el orden que espera el preprocesador.
    """
    if isinstance(data, pd.DataFrame):
        df = data.copy()
    elif isinstance(data, dict):
        df = pd.DataFrame([data])
    else:
        # lista de dicts o algo parecido
        df = pd.DataFrame(list(data))

    if RAW_COLS is None:
        # si no tenemos RAW_COLS, usamos lo que venga (pero es menos seguro)
        return df

    # Asegurar columnas: crear faltantes y respetar orden
    for c in RAW_COLS:
        if c not in df.columns:
            df[c] = np.nan
    # eliminar columnas desconocidas para evitar errores
    df = df[RAW_COLS]
    return df

# ---------- predicción por lotes ----------
def predict_batch(records, threshold=0.5):
    """
    records: dict o list[dict] o DataFrame (crudo)
    threshold: umbral de decisión para convertir probas en 0/1
    """
    df_raw = to_aligned_dataframe(records)
    X = preprocessor.transform(df_raw)            # -> matriz transformada (numpy/scipy)
    proba = clf.predict_proba(X)[:, 1]            # probabilidad de clase 1 (Churn)
    pred  = (proba >= threshold).astype(int)

    out = df_raw.copy()
    out["proba_churn"] = proba
    out["pred"] = pred
    return out

# ---------- ejemplo rápido ----------
# Ejemplo con un dict (ajústalo a tus columnas crudas reales)
# record = {
#     "gender": "Female",
#     "SeniorCitizen": 0,
#     "Partner": "Yes",
#     "Dependents": "No",
#     "tenure": 12,
#     "PhoneService": "Yes",
#     "MultipleLines": "No",
#     "InternetService": "Fiber optic",
#     "OnlineSecurity": "No",
#     "OnlineBackup": "Yes",
#     "DeviceProtection": "No",
#     "TechSupport": "No",
#     "StreamingTV": "Yes",
#     "StreamingMovies": "Yes",
#     "Contract": "Month-to-month",
#     "PaperlessBilling": "Yes",
#     "PaymentMethod": "Electronic check",
#     "MonthlyCharges": 79.85,
#     "TotalCharges": 332.3
# }
# print(predict_batch(record))

# Si quieres probar con varios:
# print(predict_batch([record, record]))



PROJECT_ROOT: c:\Users\juana\MLOPS\miniproyecto6\docs
MODELS_DIR  : C:\Users\juana\MLOPS\miniproyecto6\docs\models
RAW_COLS (del preprocessor): 20
TRANSFORMED_NAMES: 45
